# Sales Account Meeting Briefing

**User Story**: US1 — Meeting Preparation Assistant
**Persona**: Account Executive

This notebook demonstrates how to use the Salesforce AI Assistant to prepare
for account meetings. The assistant pulls live CRM data (account details, contacts,
open opportunities, recent activities) and generates a comprehensive briefing
with recommended talking points.

## Prerequisites
- `.env` file configured with Salesforce and Azure AI credentials
- `salesforce-crm` MCP server available
- Python packages installed (`pip install -r requirements.txt`)

In [ ]:
# Cell 2: Environment + Auth Setup
import os
import sys

# Add project root to path
sys.path.insert(0, os.path.abspath(".."))

from dotenv import load_dotenv

load_dotenv("../.env")

# Verify required env vars
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing environment variables: {', '.join(missing)}")

print("✅ Environment configured")
print(f"   Project: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:50]}...")
print(f"   SF Instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: MCP Connection — salesforce-crm
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Connect to salesforce-crm MCP server via stdio
mcp_connection = project.connections.get("salesforce-crm")
print(f"✅ MCP connection: {mcp_connection.name}")

In [ ]:
# Cell 4: Create Sales Agent
from azure.ai.projects.models import McpToolConnection

# Load system prompt
with open("../agents/sales/system_prompt.md") as f:
    system_prompt = f.read()

# Create MCP toolset
mcp_tool = McpToolConnection(connection=mcp_connection)

# Create the Sales Agent
agent = project.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name="Sales Meeting Prep Agent",
    instructions=system_prompt,
    tools=[mcp_tool],
)
print(f"✅ Agent created: {agent.name} (ID: {agent.id})")

# Create a conversation thread
thread = project.agents.threads.create()
print(f"✅ Thread created: {thread.id}")

In [ ]:
# Cell 5: Meeting Briefing Request
ACCOUNT_NAME = "Acme Corp"  # Change to your target account

message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Prepare me for my meeting with {ACCOUNT_NAME}. "
    f"Include account overview, key contacts, open opportunities, "
    f"recent activities, and suggested talking points.",
)

# Run the agent
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)
print(f"✅ Run completed: {run.status}")

In [ ]:
# Cell 6: Display Briefing
from IPython.display import Markdown, display

# Get the agent's response
messages = project.agents.messages.list(thread_id=thread.id)

# Display the latest assistant message
for msg in messages.data:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, "text"):
                display(Markdown(content.text.value))
        break  # Only show the latest

In [ ]:
# Cell 7: Follow-up Question
follow_up = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Any other open opportunities with them? What's the total pipeline value?",
)

run2 = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

# Display follow-up response
messages2 = project.agents.messages.list(thread_id=thread.id)
for msg in messages2.data:
    if msg.role == "assistant":
        for content in msg.content:
            if hasattr(content, "text"):
                display(Markdown(content.text.value))
        break

In [ ]:
# Cell 8: Cleanup
project.agents.delete_agent(agent.id)
print(f"✅ Agent deleted: {agent.id}")
print("\n--- Session Complete ---")